In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords

In [4]:
s=pd.read_csv('/content/drive/MyDrive/NLP/train.csv')
st=pd.DataFrame(s)
st.head()

,id,qid1,qid2,question1,question2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [5]:
st.shape

(404290, 5)

In [6]:
st.isnull().sum()

id           0
qid1         0
qid2         0
question1    1
question2    2
dtype: int64

In [7]:
st.dropna(inplace=True)

In [9]:
qids = pd.Series(st['qid1'].tolist() + st['qid2'].tolist())
unique_qs = len(np.unique(qids))
repeated_qs = np.sum(qids.value_counts() > 1)
print("Total no. unique questions: ", unique_qs)
print("Toatal no. of repeated questios: ", repeated_qs)

Total no. unique questions:  537929
Toatal no. of repeated questios:  111778


In [10]:
st['freq_qid1'] = st.groupby('qid1')['qid1'].transform('count')
st['freq_qid2'] = st.groupby('qid2')['qid2'].transform('count')
st['q1len'] = st['question1'].str.len()
st['q2len'] = st['question2'].str.len()
st['q1_n_words'] = st['question1'].apply(lambda row: len(row.split(" ")))
st['q2_n_words'] = st['question2'].apply(lambda row: len(row.split(" ")))

In [11]:
def normalized_common_words(row):
  w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
  w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
  return 1.0 * len(w1 & w2)

st['word_Common'] = st.apply(normalized_common_words, axis= 1)

def normalized_total_words(row):
  w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
  w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
  return 1.0 * (len(w1) + len(w2))

st['word_Total'] = st.apply(normalized_total_words, axis= 1)

def normalized_shared_words(row):
  w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
  w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
  return 1.0 * len(w1 & w2) / (len(w1) + len(w2))

In [12]:
st['word_Total'] = st.apply(normalized_total_words, axis= 1)

def normalized_shared_words(row):
  w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
  w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
  return 1.0 * len(w1 & w2) / (len(w1) + len(w2))

st['word_share'] = st.apply(normalized_shared_words, axis= 1)

st['freq_q1+q2'] = st['freq_qid1'] + st['freq_qid2']
st['freq_q1-q2'] = st['freq_qid1'] - st['freq_qid2']

st.head()

,id,qid1,qid2,question1,question2,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,3,1,76,39,13,7,2.0,20.0,0.100000,4,2
